In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import re
import plotly.express as px
import ast
from geopy.geocoders import Nominatim



In [3]:
df=pd.read_csv('items.csv')

In [4]:
df.columns

Index(['series', 'identifier_swisscollections', 'subject_form',
       'library_call_number', 'resource_type',
       'date_of_publication_or_production_standardised', 'place_standardised',
       'physical_description', 'language', 'title'],
      dtype='object')

In [5]:
df.head()

,series,identifier_swisscollections,subject_form,library_call_number,resource_type,date_of_publication_or_production_standardised,place_standardised,physical_description,language,title
0,"['Autographensammlung Karl Geigy-Hagenbach. I,...",991170522224305501,"['Autograf', 'Handschrift', 'Briefsammlung']","Basel, UB, UBH Autogr Geigy-Hagenbach 1127","['Brief', 'Autograph', 'Archivmaterial / Archi...",18060815,"{'name': 'Kopenhagen', 'identifier': ['(DE-588...","['2 Bl. (2 S. beschrieben) ; 19,5 x 12,5 cm']",['Deutsch'],Brief an Breitkopf und Härtel
1,"['Autographensammlung Karl Geigy-Hagenbach. I,...",991170522224905501,"['Autograf', 'Handschrift', 'Briefsammlung']","Autographensammlung Geigy-Hagenbach, -","['Brief', 'Autograph', 'Archivmaterial / Archi...",17940115,"{'name': 'Zürich', 'identifier': ['(DE-588)406...","['2 Bl. (4 S. beschrieben) ; 22,5 x 19 cm']",['Deutsch'],Brief an [Gottlieb Hufeland]
2,"['Autographensammlung Karl Geigy-Hagenbach. I,...",991170522225205501,"['Autograf', 'Handschrift', 'Briefsammlung']","Basel, UB, UBH Autogr Geigy-Hagenbach 1121","['Brief', 'Autograph', 'Archivmaterial / Archi...",18070822,"{'name': 'Nürnberg', 'identifier': ['(DE-588)4...",['1 Bl. (2 S. beschrieben) ; 18 x 12 cm'],['Deutsch'],Brief an Hofrat
3,"['Autographensammlung Karl Geigy-Hagenbach. I,...",991170522227205501,"['Autograf', 'Handschrift', 'Briefsammlung']","Autographensammlung Geigy-Hagenbach, -","['Brief', 'Autograph', 'Archivmaterial / Archi...",17960601,"{'name': 'Mannheim', 'identifier': ['(DE-588)4...","['2 Bl. (1 S. beschrieben) ; 18,5 x 11,5 cm']",['Deutsch'],Brief an Unbekannt
4,"['Autographensammlung Karl Geigy-Hagenbach. I,...",991170522227605501,"['Autograf', 'Handschrift', 'Briefsammlung']","Autographensammlung Geigy-Hagenbach, -","['Brief', 'Autograph', 'Archivmaterial / Archi...",18061213,"{'name': 'Berlin', 'identifier': ['(DE-588)400...",['1 Bl. (2 S. beschrieben) ; 23 x 19 cm'],['Deutsch'],Brief an [Christian Gottlob] Voigt


In [6]:
df.found_data=pd.to_datetime(df.date_of_publication_or_production_standardised, errors='coerce')

C:\Users\Yasaman\AppData\Local\Temp\ipykernel_9144\2702049369.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.found_data=pd.to_datetime(df.date_of_publication_or_production_standardised, errors='coerce')


In [7]:
def find_date_of_publication(x):
    try:
        numbers_only = re.findall(r'\d+', '711, ca zwischen 1930-1972')

        return int([x for x in numbers_only if len(x) > 3][0][:4])
    except:
        print(x)
        return None


In [8]:
df.found_data=df.date_of_publication_or_production_standardised.apply(find_date_of_publication)

In [9]:
df['language'] = df['language'].str.replace("[\[\]']", '', regex=True)  # Remove brackets and quotes
df['language'] = df['language'].str.split(',\s*')  # Split by commas with optional whitespace


In [10]:
mapping_language={'English':'Englisch','England':'Englisch','Deusch':'Deutsch','Detusch':'Deutsch'  ,  "Franzsösisch": "Französisch",
    "Französich": "Französisch",
    "Franösisch": "Französisch",'French':'Französisch','German':'Deutsch','Lateinisch':'Latein','Titel: Deutsch':'Deutsch','Turkisch':'Türkisch','Urdu ?':'Urdu','deutsch':'Deutsch','französisch':'Französisch','Paris':'Französisch','Ger':'Deutsch','Russian':'Russisch','Handgeschriebenes Gedicht Schweizerdeutsch':'Schweizerdeutsch','Einleitung Deutsch':'Deutsch','Haupttext Deutsch ; Unterschrift Aleksanderʺ In Alter Russischer Schrift':'Deutsch','Dialoge Schweizerdeutsch':'Schweizerdeutsch','Deutschsprachiger Entwurf':'Deutsch','Italienische':'Italienisch','Nebst Latein':'Latein'}

In [11]:
# Function to split on "und" and replace with a list of words
def split_on_und(values):
    new_values=[]
    for value in values:
        
        value = ''.join([i for i in value if not i.isdigit()])
    # Remove extra spaces
        value = ' '.join(value.split())
        if 'Texte' in value:
            value=value.replace('Texte','')
        if 'Text' in value:
            value=value.replace('Text','')
        if ' und ' in value:
            new_list=value.split(' und ')
            for v in new_list:
                v=v.title()
                if v in mapping_language:
                 
                    new_values.append(mapping_language[v].title())
                else:
       
                    new_values.append(v.title())
        else:
            if value.title() in mapping_language:
                
                new_values+=[mapping_language[value.title()]]
            else:
                new_values+=[value.title()]
    return new_values

In [12]:
df.corrected_lang=df.language.apply(split_on_und)

C:\Users\Yasaman\AppData\Local\Temp\ipykernel_9144\3513459547.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.corrected_lang=df.language.apply(split_on_und)


In [13]:

# Step 2: Flatten the list of lists and find unique values
unique_languages = set([lang.strip() for sublist in df.corrected_lang for lang in sublist])

# Display the result as a sorted list
unique_languages = sorted(unique_languages)
for  x in unique_languages[40:]:
    print(x)

Schwedisch
Schweizerdeutsch
Schweizerdeutsch (Alte Häädler Puuresproch)
Spanisch
Tschechisch
Türkisch
Ungarisch
Urdu
Usbekisch
Vinzel


In [14]:
df['corrected_subject_form'] = df['subject_form'].str.replace("[\[\]']", '', regex=True)  # Remove brackets and quotes
df['corrected_subject_form'] = df['corrected_subject_form'].str.split(',\s*')  # Split by commas with optional whitespace

In [15]:

# Step 2: Flatten the list of lists and find unique values
unique_subjects = set([subj.strip() for sublist in df.corrected_subject_form for subj in sublist])

# Display the result as a sorted list
unique_subjects = sorted(unique_subjects)
for  x in unique_subjects[40:60]:
    print(x)

Stammbuch
Stimme
Tagebuch
Technische Zeichnung
Theaterstück
Vorlesung
Zeichnung
Zeitschrift
Zeitung


In [16]:
def get_city(x):
    if x:
        return x.split(',')[0]
    return None

In [17]:
df.library_city=df.library_call_number.apply(get_city)

C:\Users\Yasaman\AppData\Local\Temp\ipykernel_9144\234623331.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.library_city=df.library_call_number.apply(get_city)


In [18]:
location_maps={'Autographensammlung Geigy-Hagenbach':'Basel','Bernoulli-Briefinventar':'Basel'}

In [19]:
df['library_city']=df.library_city.apply(lambda x: location_maps[x] if x in location_maps else x)

In [20]:
latitudes = {
    'Basel': 47.5596,
    'Solothurn': 47.2088,
    'Trogen': 47.4084,
    'St. Gallen': 47.4239,
    'Zofingen': 47.2889,
    'Bern': 46.9481,
    'Luzern': 47.0502,
    'Aarau': 47.3925
}

longitudes = {
    'Basel': 7.5886,
    'Solothurn': 7.5323,
    'Trogen': 9.4622,
    'St. Gallen': 9.3748,
    'Zofingen': 7.9448,
    'Bern': 7.4474,
    'Luzern': 8.3093,
    'Aarau': 8.0443
}



df['location_library_latitude']=df.library_city.map(latitudes)
df['location_library_longitude']=df.library_city.map(longitudes)

In [21]:
city_plot=df.groupby(['library_city','location_library_latitude','location_library_longitude'])['series'].count().reset_index().rename(columns={'series':'count'})

In [22]:
city_plot['squared_root']=np.sqrt(city_plot['count'])

In [23]:

# Create the map plot with circles sized by 'count'
fig = px.scatter_mapbox(
    city_plot, 
    lat="location_library_latitude", 
    lon="location_library_longitude", 
    size_max=50, 
    hover_name="library_city",
    size='squared_root',
    hover_data={"count": True,'squared_root':False},
    title="Records distribution by Library's location",
    mapbox_style="open-street-map",
    zoom=6,
    center={"lat": 46.85, "lon": 8.2}
)

fig.show()

In [24]:
# Convert the 'city_info' column from string to dictionary
df['place_standardised'] = df['place_standardised'].apply(ast.literal_eval)

# Optionally, extract the 'name' and 'identifier' fields into separate columns
df['city_info'] = df['place_standardised'].apply(lambda x: x['name'])
#df['identifier'] = df['city_info'].apply(lambda x: x['identifier'])

In [25]:
from tqdm import tqdm
tqdm.pandas()

In [28]:
df_cities=pd.read_csv('worldcities.csv')

In [34]:
df_cities

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37732000.0,1392685764
1,Jakarta,Jakarta,-6.1750,106.8275,Indonesia,ID,IDN,Jakarta,primary,33756000.0,1360771077
2,Delhi,Delhi,28.6100,77.2300,India,IN,IND,Delhi,admin,32226000.0,1356872604
3,Guangzhou,Guangzhou,23.1300,113.2600,China,CN,CHN,Guangdong,admin,26940000.0,1156237133
4,Mumbai,Mumbai,19.0761,72.8775,India,IN,IND,Mahārāshtra,admin,24973000.0,1356226629
...,...,...,...,...,...,...,...,...,...,...,...
47863,Munha-dong,Munha-dong,39.3813,127.2517,"Korea, North",KP,PRK,Kangwŏn,NaN,NaN,1408979215
47864,Sil-li,Sil-li,39.4880,125.4640,"Korea, North",KP,PRK,P’yŏngnam,NaN,NaN,1408767958
47865,Muan,Muan,34.9897,126.4714,"Korea, South",KR,KOR,Jeonnam,admin,NaN,1410001061
47866,Hongseong,Hongseong,36.6009,126.6650,"Korea, South",KR,KOR,Chungnam,admin,NaN,1410822139


In [50]:
df=df.merge(df_cities[['city','lat','lng','country']], right_on='city',left_on='city_info', how='left')

In [51]:
df

,series,identifier_swisscollections,subject_form,library_call_number,resource_type,date_of_publication_or_production_standardised,place_standardised,physical_description,language,title,corrected_subject_form,library_city,location_library_latitude,location_library_longitude,city_info,city,lat,lng,country
0,"['Autographensammlung Karl Geigy-Hagenbach. I,...",991170522224305501,"['Autograf', 'Handschrift', 'Briefsammlung']","Basel, UB, UBH Autogr Geigy-Hagenbach 1127","['Brief', 'Autograph', 'Archivmaterial / Archi...",18060815,"{'name': 'Kopenhagen', 'identifier': ['(DE-588...","['2 Bl. (2 S. beschrieben) ; 19,5 x 12,5 cm']",[Deutsch],Brief an Breitkopf und Härtel,"[Autograf, Handschrift, Briefsammlung]",Basel,47.5596,7.5886,Kopenhagen,NaN,NaN,NaN,NaN
1,"['Autographensammlung Karl Geigy-Hagenbach. I,...",991170522224905501,"['Autograf', 'Handschrift', 'Briefsammlung']","Autographensammlung Geigy-Hagenbach, -","['Brief', 'Autograph', 'Archivmaterial / Archi...",17940115,"{'name': 'Zürich', 'identifier': ['(DE-588)406...","['2 Bl. (4 S. beschrieben) ; 22,5 x 19 cm']",[Deutsch],Brief an [Gottlieb Hufeland],"[Autograf, Handschrift, Briefsammlung]",Basel,NaN,NaN,Zürich,Zürich,47.3744,8.5411,Switzerland
2,"['Autographensammlung Karl Geigy-Hagenbach. I,...",991170522225205501,"['Autograf', 'Handschrift', 'Briefsammlung']","Basel, UB, UBH Autogr Geigy-Hagenbach 1121","['Brief', 'Autograph', 'Archivmaterial / Archi...",18070822,"{'name': 'Nürnberg', 'identifier': ['(DE-588)4...",['1 Bl. (2 S. beschrieben) ; 18 x 12 cm'],[Deutsch],Brief an Hofrat,"[Autograf, Handschrift, Briefsammlung]",Basel,47.5596,7.5886,Nürnberg,NaN,NaN,NaN,NaN
3,"['Autographensammlung Karl Geigy-Hagenbach. I,...",991170522227205501,"['Autograf', 'Handschrift', 'Briefsammlung']","Autographensammlung Geigy-Hagenbach, -","['Brief', 'Autograph', 'Archivmaterial / Archi...",17960601,"{'name': 'Mannheim', 'identifier': ['(DE-588)4...","['2 Bl. (1 S. beschrieben) ; 18,5 x 11,5 cm']",[Deutsch],Brief an Unbekannt,"[Autograf, Handschrift, Briefsammlung]",Basel,NaN,NaN,Mannheim,Mannheim,49.4878,8.4661,Germany
4,"['Autographensammlung Karl Geigy-Hagenbach. I,...",991170522227605501,"['Autograf', 'Handschrift', 'Briefsammlung']","Autographensammlung Geigy-Hagenbach, -","['Brief', 'Autograph', 'Archivmaterial / Archi...",18061213,"{'name': 'Berlin', 'identifier': ['(DE-588)400...",['1 Bl. (2 S. beschrieben) ; 23 x 19 cm'],[Deutsch],Brief an [Christian Gottlob] Voigt,"[Autograf, Handschrift, Briefsammlung]",Basel,NaN,NaN,Berlin,Berlin,52.5200,13.4050,Germany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119328,"['Nachlass Gustav Teichmüller (1832-1888). B, ...",991170446014805501,"['Handschrift', 'Briefsammlung']","Basel, UB, UBH NL 79 : B 454-459","['Brief', 'Archivmaterial / Dossier']",18591114/18600731,"{'name': 'Sankt Petersburg', 'identifier': ['(...",['8 Bl.'],[Deutsch],6 Briefe an Gustav Teichmüller,"[Handschrift, Briefsammlung]",Basel,47.5596,7.5886,Sankt Petersburg,NaN,NaN,NaN,NaN
119329,"['Nachlass Gustav Teichmüller (1832-1888). B, ...",991170446015605501,"['Handschrift', 'Briefsammlung']","Basel, UB, UBH NL 79 : B 422a-423","['Brief', 'Archivmaterial / Dossier']",18641007/18690830,{'name': 'Joala'},['8 Bl.'],[Deutsch],3 Briefe an Gustav Teichmüller,"[Handschrift, Briefsammlung]",Basel,47.5596,7.5886,Joala,NaN,NaN,NaN,NaN
119330,"['Nachlass Gustav Teichmüller (1832-1888). B, ...",991170446017505501,"['Handschrift', 'Briefsammlung']","Basel, UB, UBH NL 79 : B 96-97","['Brief', 'Archivmaterial / Dossier']",18600325/18670217,{'name': 'Plön'},['2 Bl.'],[Deutsch],2 Briefe an Gustav Teichmüller,"[Handschrift, Briefsammlung]",Basel,47.5596,7.5886,Plön,Plön,54.1622,10.4214,Germany
119331,"['Nachlass Gustav Teichmüller (1832-1888). B, ...",991170446017805501,"['Handschrift', 'Briefsammlung']","Basel, UB, UBH NL 79 : B 93",['Brief'],1857,"{'name': 'Tartu', 'identifier': ['(DE-588)1029...",['1 Bl. ; 13 x 21 cm'],[Deutsch],Brief an Behm,"[Handschrif

In [ ]:
# Define function to get latitude and longitude
geolocator = Nominatim(user_agent='myapplication')
def get_lat_long(city_name):

    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None
cities=df[df.city.isna()]['city_info'].unique()
city_mapping={city:get_lat_long(city) for city in tqdm(cities)}

  1%|          | 26/4194 [00:27<39:11,  1.77it/s]  

100%|██████████| 10/10 [00:07<00:00,  1.27it/s]


In [ ]:
city_mapping

{'Kopenhagen': (55.6867243, 12.5700724),
 'Zürich': (47.3744489, 8.5410422),
 'Nürnberg': (49.453872, 11.077298),
 'Mannheim': (49.4892913, 8.4673098),
 'Berlin': (52.510885, 13.3989367),
 'Sankt Petersburg': (59.938732, 30.316229),
 'Karlsbad': (50.2306216, 12.8701437),
 'Elzach': (48.1757263, 8.0718474),
 'Karlsruhe': (49.0068705, 8.4034195),
 'Stuttgart': (48.7784485, 9.1800132)}

In [ ]:

# Apply function to the DataFrame column
df[['place_latitude', 'place_longitude']] = df['city_info'].progress_apply(lambda x: pd.Series(get_lat_long(x)))

-87.6244212


In [ ]:
df['place_standardised'][4]

"{'name': 'Berlin', 'identifier': ['(DE-588)4005728-8']}"

In [ ]:
df.library_call_number

0         Basel, UB, UBH Autogr Geigy-Hagenbach 1127
1             Autographensammlung Geigy-Hagenbach, -
2         Basel, UB, UBH Autogr Geigy-Hagenbach 1121
3             Autographensammlung Geigy-Hagenbach, -
4             Autographensammlung Geigy-Hagenbach, -
                             ...                    
100012              Basel, UB, UBH NL 79 : B 454-459
100013             Basel, UB, UBH NL 79 : B 422a-423
100014                Basel, UB, UBH NL 79 : B 96-97
100015                   Basel, UB, UBH NL 79 : B 93
100016                Basel, UB, UBH NL 79 : B 87-88
Name: library_call_number, Length: 100017, dtype: object